In [2]:
import tensorflow as tf

In [1]:
class ModelGen:
    def __init__(self):
        pass
    def generateModel(self, layers, session, numberOfCurrencies, timeFrame=50):
        self._s = session
        self.finalModel = []
        self.finalModel.append(tf.placeholder(shape=[None, numberOfCurrencies, timeFrame, 4], dtype=tf.float32))
        layerCount = 0
        for layer in layers:
            if layer["type"] == 'conv':            
                self.finalModel.append(tf.layers.conv(inputs=finalModel[layerCount], filters=layer["filters"], kernel_size = [1,layer["size"]], activation=layer["activation"]))
            elif layer["type"] == 'dense':
                self.finalModel.append(tf.layers.dense(inputs=finalModel[layerCount], layer["size"], activation=layer["activation"]))
        self.finalModel.append(tf.nn.softmax(self.finalModel[-1], axis=1))
        
        self.priceChanges = tf.placeholder(shape=[None, numberOfCurrencies, 1], dtype=tf.float32)
        self._allocate = finalModel[-1]
        
        self.averageLoss = tf.reduce_mean(tf.matmul(tf.matrix_transpose(self.priceChanges), 
                                             tf.scalar_mul(tf.constant(initialPortfolio), 
                                               tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))))

        self.loss = tf.exp(tf.reduce_sum(tf.multiply(-tf.log(self.priceChanges), tf.reshape(self._allocate, [tf.shape(self._allocate)[0], numberOfCurrencies, 1]))))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        self._train = self.optimizer.minimize(self.loss)
        
    def act(self, observation):
        return self._s.run(self._allocate, feed_dict={self.inputT: observation})
    
    def train_step(self, obs, prices):
        batch_feed = {self.inputT : obs,
                     self.priceChanges: prices
                     }
        _, lossValue = self._s.run([self._train, self.averageLoss], feed_dict=batch_feed)
        return lossValue
    def test_model(self, obs, prices):
        print(self._s.run(self.averageLoss, feed_dict={self.inputT : obs, self.priceChanges: prices}))
        print(prices)
        